In [1]:
from utils.arguments import parse_args
from utils.dataloader import DatasetQuickdraw
import utils.misc as misc
from algorithms import create_model
import torch.optim as optim
import torch
import os

In [2]:
import numpy as np
import os

import matplotlib.pyplot as plt


In [3]:
misc.seed_all(0)
transform = None
dataset_train = DatasetQuickdraw('quickdraw_png', transform, mode="train")
dataset_val = DatasetQuickdraw('quickdraw_png', transform, mode="test")

Data loading:   4%|▍         | 1/25 [00:00<00:03,  6.96it/s]

Data: Loaded 1750000 samples


Data loading: 100%|██████████| 25/25 [00:01<00:00, 16.98it/s]


Data: Loaded 62500 samples


In [4]:
class SVM:
    def __init__(self,C=1.0):
        self.C = C
        self.W = 0
        self.b = 0
        
    def hingeLoss(self,W,b,X,Y):
        loss  = 0.0
        
        loss += .5*np.dot(W,W.T)
        
        m = X.shape[0]
        
        for i in range(m):
            ti = Y[i]*(np.dot(W,X[i].T)+b)
            loss += self.C *max(0,(1-ti))
            
        return loss[0][0]
    
    def fit(self,X,Y,batch_size=50,learning_rate=0.001,maxItr=500):
        
        no_of_features = X.shape[1]
        no_of_samples = X.shape[0]
        
        n = learning_rate
        c = self.C
        
        #Init the model parameters
        W = np.zeros((1,no_of_features))
        bias = 0
        
        #Initial Loss
        
        #Training from here...
        # Weight and Bias update rule
        losses = []
        
        for i in range(maxItr):
            #Training Loop
            
            l = self.hingeLoss(W,bias,X,Y)
            losses.append(l)
            ids = np.arange(no_of_samples)
            np.random.shuffle(ids)
            
            #Batch Gradient Descent(Paper) with random shuffling
            for batch_start in range(0,no_of_samples,batch_size):
                #Assume 0 gradient for the batch
                gradw = 0
                gradb = 0
                
                #Iterate over all examples in the mini batch
                for j in range(batch_start,batch_start+batch_size):
                    if j<no_of_samples:
                        i = ids[j]
                        ti =  Y[i]*(np.dot(W,X[i].T)+bias)
                        
                        if ti>1:
                            gradw += 0
                            gradb += 0
                        else:
                            gradw += c*Y[i]*X[i]
                            gradb += c*Y[i]
                            
                #Gradient for the batch is ready! Update W,B
                W = W - n*W + n*gradw
                bias = bias + n*gradb
                
        
        self.W = W
        self.b = bias
        return W,bias,losses

In [5]:
data_train = np.array(dataset_train.data, dtype='float32')/255.0
label_train= np.array(dataset_train.labels)
data_test = np.array(dataset_val.data, dtype='float32')/255.0
label_test = np.array(dataset_val.labels)

In [6]:
import random
combined = list(zip(data_train, label_train))
random.shuffle(combined)
data_train[:], label_train[:] = zip(*combined)

In [7]:
combine = list(zip(data_test, label_test))
random.shuffle(combine)
data_test[:], label_test[:] = zip(*combine)

In [8]:
M = data_train.shape[0]
data_train = data_train.reshape(M,-1)
print(data_train.shape)
print(label_train.shape)

(1750000, 784)
(1750000,)


In [9]:
N = data_test.shape[0]
data_test = data_test.reshape(N, -1)
print(data_test.shape)
print(label_test.shape)

(62500, 784)
(62500,)


In [10]:
number_of_classes = len(np.unique(label_train))

In [11]:
def classWiseData(x, y):
    data = {}
    
    for i in range(number_of_classes):
        data[i] = []
        
    for i in range(x.shape[0]):
        data[y[i]].append(x[i])
        
    for k in data.keys():
        data[k] = np.array(data[k])
        
    return data

In [12]:
data = classWiseData(data_train, label_train)

In [13]:
print(data[0].shape[0])
print(data[1].shape[0])

70000
70000


In [14]:
def getDataPairForSVM(d1,d2):
    
    l1,l2 = d1.shape[0], d2.shape[0]
    samples = l1+l2
    features = d1.shape[1]
    
    data_pair = np.zeros((samples,features))
    data_labels = np.zeros((samples,))
    
    data_pair[:l1,:] = d1
    data_pair[l1:,:] = d2
    
    data_labels[:l1] = -1
    data_labels[l1:] = 1
    
    return data_pair, data_labels

In [ ]:
mySVM = SVM()
xp, yp = getDataPairForSVM(data[0], data[1])
w,b,loss = mySVM.fit(xp,yp,learning_rate=0.00001,maxItr=1000)
plt.plot(loss)

In [ ]:
def trainSVMs(x,y):
    svm_classifiers = {}
    
    for i in range(number_of_classes):
        svm_classifiers[i] = {}
        for j in range(i+1, number_of_classes):
            xpair,ypair = getDataPairForSVM(data[i],data[j])
            wts,b,loss = mySVM.fit(xpair, ypair,learning_rate=0.00001,maxItr=1000)
            svm_classifiers[i][j] = (wts,b)
            
            plt.plot(loss)
            plt.savefig('{}_{}.png'.format(i,j))
            plt.show()
            
    return svm_classifiers

In [ ]:
svm_classifiers = trainSVMs(data_train, label_train)

In [ ]:
print('{}_{}.png'.format(1,2))